In [1]:
def corr_kg_fob_via_preço(imp_fert,z=3):

    col=imp_fert.columns

    col_quanti=['kg','US$_fob']
    
    # Preço FOB e do Frete
    for q in col_quanti:
        imp_fert.insert(len(imp_fert.columns),'log_'+q,np.log10(imp_fert[q]))
    col_quanti
    
    ### z_score
    
    col_log=imp_fert.columns[-2:]
    col_log
    
    def z_score(imp_fert,l):
        média=imp_fert[l].mean()
        k=len(imp_fert.columns)
        imp_fert.insert(k,'mean_'+l,média)
        imp_fert.insert(k+1,'qd_dif'+l,(imp_fert[l]-imp_fert['mean_'+l])**2)
        imp_fert.insert(k+2,'std_'+l, (imp_fert['qd_dif'+l].sum()/len(imp_fert))**(1/2))
        imp_fert.insert(k+3,'z_score_'+l, (imp_fert[l]-imp_fert['mean_'+l])/imp_fert['std_'+l])
        
        return imp_fert
    
    for l in col_log:
        imp_fert=z_score(imp_fert,l)
        
    
    k=len(imp_fert.columns)
    imp_fert.insert(k,
                    'dif_z_score',
                    imp_fert['z_score_'+col_log[0]] - imp_fert['z_score_'+col_log[1]]
                   )
    
    imp_fert=z_score(imp_fert,'dif_z_score')
    
    borda=3
    
    imp_fert_corr=imp_fert[np.abs(imp_fert.z_score_dif_z_score)<z]
    imp_fert_err=imp_fert[np.abs(imp_fert.z_score_dif_z_score)>z]
    
    preço=imp_fert_corr['US$_fob'].sum()/imp_fert_corr.kg.sum()
    
    imp_fert_err=imp_fert_err.assign(preço=preço)
    
    imp_fert_err=imp_fert_err.assign(kg_err=1)
    
    imp_fert_err.loc[np.abs(imp_fert_err['z_score_log_US$_fob'])>=np.\
                     abs(imp_fert_err.z_score_log_kg),'kg_err']=0
    
    imp_fert_err_kg=imp_fert_err[imp_fert_err.kg_err==1]
    
    imp_fert_err_fob=imp_fert_err[imp_fert_err.kg_err==0]
    
    imp_fert_err_kg.kg=imp_fert_err_kg['US$_fob']/preço

    len_err_kg=len(imp_fert_err_kg)
    
    imp_fert_err_fob['US$_fob']=imp_fert_err_fob.kg*preço
    
    len_err_fob=len(imp_fert_err_fob)
    
    imp_fert=pd.concat([imp_fert_corr,imp_fert_err_kg,imp_fert_err_fob])
    
    imp_fert=imp_fert[col]
    
    return imp_fert, len_err_kg, len_err_fob

print('** Função corr_kg_fob_via_preço(imp_fert,z=3) disponível **')

** Função corr_kg_fob_via_preço(imp_fert,z=3) disponível **
